In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt

import readdata as rd

import datetime
sys.path.append('/Users/ajc/Desktop/ATS/ats-repo/ats/tools/utils/')

In [3]:
outfile = '/Users/ajc/AllPostProcessData/BarrowTransect/NGEE/Temperature-SiteA/'

In [17]:
infile = '/Users/ajc/Desktop/ATS/barrow-polygon-data/Vladimir-Data/Area_A/'
sim_num = 5
simulation = 'Arun%sR22'%sim_num
infile_simulated11 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun%sR21/'%sim_num
infile_simulated12 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/%s/'%simulation
save = False
print infile_simulated11
start_year = 2013
if start_year == 2013:
    print start_year
    dataset= 'NGEE_BRW_A_2012-09-01_2013-09-30_Thermal_Transect.csv'
elif start_year == 2014:
    dataset= 'NGEE_BRW_A_2013-10-01_2014-09-30_Thermal_Transect_and_Borehole.csv'

infile_observed = infile + dataset
data = pd.read_csv(infile_observed, skiprows=5)
#print data.columns
DataObservedTrough = dict()
DataObservedCenter = dict()
DataObservedRim = dict()

#origin=pd.Timestamp('1960-01-01')
d = dict()
time_origin = datetime.datetime(start_year,1,1,0,0,0)
d['time'] = [pd.to_datetime(data['Timestamp'])]


Time = [(t - time_origin).dt.total_seconds()/86400 for t in d[u'time']]
Time = np.array([t for t in Time[0]])
#print Time[0]
start_time = np.where(Time == 0)[0]
print start_time
start_time = start_time[0]

for c in data.columns:
    if 'trough' in c:
        k = int(float(c[6:10])*100)
        d = np.array([x + 273.15  for x in  data[c]])
        DataObservedTrough['temperature-%scm'%k] = d
    elif 'center' in c and not 'off' in c:
        k = int(float(c[6:10])*100)
        #print k, c
        d = np.array([ x + 273.15  for x in data[c]])
        #d = np.array([  d1[0] if np.isnan(x) else x  for i,x in enumerate(d1)])
        
        DataObservedCenter['temperature-%scm'%k] = d
    elif 'rim' in c and 'rim2' not in c:
        k = int(float(c[6:10])*100)
        d = np.array([x + 273.15  for x in data[c]])
        if k != 2 and start_year == 2014:
            k = k + 4
        #print k, c
        DataObservedRim['temperature-%scm'%k] = d
DataObservedCenter['time'] = Time  
DataObservedTrough['time'] = Time  
DataObservedRim['time'] = Time  
for key, d in DataObservedCenter.iteritems():
    DataObservedCenter[key] = np.array(DataObservedCenter[key])
for key, d in DataObservedTrough.iteritems():
    DataObservedTrough[key] = np.array(DataObservedTrough[key])
for key, d in DataObservedRim.iteritems():
    DataObservedRim[key] = np.array(DataObservedRim[key])

/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun5R21/
2013
[2735]


In [18]:

depth = ['2cm','5cm', '10cm', '15cm', '20cm', '25cm', '30cm', '35cm', '40cm', '50cm', '60cm', '70cm', '80cm', '100cm', '125cm', '150cm']


DataSim11Trough = dict()
DataSim11Center = dict()
DataSim11Rim = dict()
TimeSim11 = []
for d in depth:
    file = infile_simulated11 + 'temperature-center-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim11Center['temperature-%s'%d] = data['data']

    file = infile_simulated11 + 'temperature-trough-%s.dat'%d
    data = rd.ReadSingleFile(file)
    #print data
    #break
    DataSim11Trough['temperature-%s'%d] = data['data']
    
    file = infile_simulated11 + 'temperature-rim-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim11Rim['temperature-%s'%d] = data['data']
    
TimeSim11 = [t/86400. for t in data['time']]

#print DataSimTrough['temperature-100cm'][:2]

In [20]:

depth = ['2cm','5cm', '10cm', '15cm', '20cm', '25cm', '30cm', '35cm', 
         '40cm', '50cm', '60cm', '70cm', '80cm', '100cm', '125cm', '150cm']

DataSim12Trough = dict()
DataSim12Center = dict()
DataSim12Rim = dict()
TimeSim12 = []
for d in depth:
    file = infile_simulated12 + 'temperature-center-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim12Center['temperature-%s'%d] = data['data']

    file = infile_simulated12 + 'temperature-trough-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim12Trough['temperature-%s'%d] = data['data']
    
    file = infile_simulated12 + 'temperature-rim-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim12Rim['temperature-%s'%d] = data['data']
    
TimeSim12 = [t/86400. for t in data['time']]

#print DataObservedCenter['temperature-100cm'][:2]
#print DataObservedTrough['temperature-100cm'][:2]
#print DataObservedRim['temperature-100cm'][:20]

In [21]:
def put_axis(dep, loc=''):
    plt.xlabel('Day (%s)'%start_year,fontsize=12, fontweight='normal')
    plt.ylabel('Temperature [K]',fontsize=12, fontweight='normal')
    plt.text(4, 285, '-%s, %s, %s'%(dep,start_year, loc),fontsize=12, fontweight='normal')
    plt.ylim((240, 290))
    plt.yticks(np.linspace(245,290, 6))
    plt.xlim((0, 280))
    plt.xticks(np.linspace(0,280, 5))
    plt.axhline(y=273.25,color='b')

DataObservedCenterDaily = dict()
DataObservedTroughDaily = dict()
DataObservedRimDaily = dict()
DataSim11TroughDaily = dict()
DataSim11CenterDaily = dict()
DataSim11RimDaily = dict()
DataSim12TroughDaily = dict()
DataSim12CenterDaily = dict()
DataSim12RimDaily = dict()

l = len(DataObservedCenter['temperature-2cm'])
r = l%24

#print np.reshape(DataObservedCenter['temperature-100cm'][:-r], (-1, 24)).mean(axis=1)
for key, d in DataObservedCenter.iteritems():
    DataObservedCenterDaily[key] = np.reshape(DataObservedCenter[key][:-r], (-1, 24)).mean(axis=1)
for key, d in DataObservedTrough.iteritems():
    DataObservedTroughDaily[key] = np.reshape(DataObservedTrough[key][:-r], (-1, 24)).mean(axis=1)
for key, d in DataObservedRim.iteritems():
    DataObservedRimDaily[key] = np.reshape(DataObservedRim[key][:-r], (-1, 24)).mean(axis=1)
    

In [22]:
# Polygon Center
fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()

plt.subplot(331)
plt.plot(Time[start_time:], DataObservedCenter['temperature-5cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-5cm'],'g-')
plt.plot(TimeSim11, DataSim11Center['temperature-5cm'],'k')
plt.plot(TimeSim12, DataSim12Center['temperature-5cm'],'k')
put_axis('5cm', 'center')

plt.subplot(332)
plt.plot(Time[start_time:], DataObservedCenter['temperature-10cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-10cm'],'g-')
plt.plot(TimeSim11, DataSim11Center['temperature-10cm'],'k')
plt.plot(TimeSim12, DataSim12Center['temperature-10cm'],'k')
put_axis('10cm', 'center')

plt.subplot(333)
plt.plot(Time[start_time:], DataObservedCenter['temperature-20cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-20cm'],'g-')
plt.plot(TimeSim11, DataSim11Center['temperature-20cm'],'k')
plt.plot(TimeSim12, DataSim12Center['temperature-20cm'],'k')
put_axis('20cm', 'center')

plt.subplot(334)
plt.plot(Time[start_time:], DataObservedCenter['temperature-40cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-40cm'],'g-')
plt.plot(TimeSim11, DataSim11Center['temperature-40cm'],'k')
plt.plot(TimeSim12, DataSim12Center['temperature-40cm'],'k')
put_axis('40cm', 'center')

plt.subplot(335)
plt.plot(Time[start_time:], DataObservedCenter['temperature-60cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-60cm'],'g-')
plt.plot(TimeSim11, DataSim11Center['temperature-60cm'],'k')
plt.plot(TimeSim12, DataSim12Center['temperature-60cm'],'k')
put_axis('60cm', 'center')

plt.subplot(336)
plt.plot(Time[start_time:], DataObservedCenter['temperature-80cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-80cm'],'g-')
plt.plot(TimeSim11, DataSim11Center['temperature-80cm'],'k')
plt.plot(TimeSim12, DataSim12Center['temperature-80cm'],'k')
put_axis('80cm', 'center')

plt.subplot(337)
plt.plot(Time[start_time:], DataObservedCenter['temperature-100cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-100cm'],'g-')
plt.plot(TimeSim11, DataSim11Center['temperature-100cm'],'k')
plt.plot(TimeSim12, DataSim12Center['temperature-100cm'],'k')
put_axis('100cm', 'center')

plt.subplot(338)
plt.plot(Time[start_time:], DataObservedCenter['temperature-125cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-125cm'],'g-')
plt.plot(TimeSim11, DataSim11Center['temperature-125cm'],'k')
plt.plot(TimeSim12, DataSim12Center['temperature-125cm'],'k')
put_axis('125cm', 'center')

plt.subplot(339)
plt.plot(Time[start_time:], DataObservedCenter['temperature-150cm'][start_time:],'r',label='Observed (hourly)')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-150cm'],'g',label='Observed (daily)')
plt.plot(TimeSim11, DataSim11Center['temperature-150cm'],'k',label='Simulated')
plt.plot(TimeSim12, DataSim12Center['temperature-150cm'],'k')
put_axis('150cm', 'center')

plt.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))

plt.tight_layout()
if save == True:
    plt.savefig(outfile+'temp-center%s-%s.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

In [23]:
# Polygon Trough
fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()

plt.subplot(331)
plt.plot(Time[start_time:], DataObservedTrough['temperature-5cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-5cm'],'g-')
plt.plot(TimeSim11, DataSim11Trough['temperature-5cm'],'k')
plt.plot(TimeSim12, DataSim12Trough['temperature-5cm'],'k')
put_axis('5cm', 'Trough')

plt.subplot(332)
plt.plot(Time[start_time:], DataObservedTrough['temperature-10cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-10cm'],'g-')
plt.plot(TimeSim11, DataSim11Trough['temperature-10cm'],'k')
plt.plot(TimeSim12, DataSim12Trough['temperature-10cm'],'k')
put_axis('10cm', 'Trough')

plt.subplot(333)
plt.plot(Time[start_time:], DataObservedTrough['temperature-20cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-20cm'],'g-')
plt.plot(TimeSim11, DataSim11Trough['temperature-20cm'],'k')
plt.plot(TimeSim12, DataSim12Trough['temperature-20cm'],'k')
put_axis('20cm', 'Trough')

plt.subplot(334)
plt.plot(Time[start_time:], DataObservedTrough['temperature-40cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-40cm'],'g-')
plt.plot(TimeSim11, DataSim11Trough['temperature-40cm'],'k')
plt.plot(TimeSim12, DataSim12Trough['temperature-40cm'],'k')
put_axis('40cm', 'Trough')

plt.subplot(335)
plt.plot(Time[start_time:], DataObservedTrough['temperature-60cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-60cm'],'g-')
plt.plot(TimeSim11, DataSim11Trough['temperature-60cm'],'k')
plt.plot(TimeSim12, DataSim12Trough['temperature-60cm'],'k')
put_axis('60cm', 'Trough')

plt.subplot(336)
plt.plot(Time[start_time:], DataObservedTrough['temperature-80cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-80cm'],'g-')
plt.plot(TimeSim11, DataSim11Trough['temperature-80cm'],'k')
plt.plot(TimeSim12, DataSim12Trough['temperature-80cm'],'k')
put_axis('80cm', 'Trough')

plt.subplot(337)
plt.plot(Time[start_time:], DataObservedTrough['temperature-100cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-100cm'],'g-')
plt.plot(TimeSim11, DataSim11Trough['temperature-100cm'],'k')
plt.plot(TimeSim12, DataSim12Trough['temperature-100cm'],'k')
put_axis('100cm', 'Trough')

plt.subplot(338)
plt.plot(Time[start_time:], DataObservedTrough['temperature-125cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-125cm'],'g-')
plt.plot(TimeSim11, DataSim11Trough['temperature-125cm'],'k')
plt.plot(TimeSim12, DataSim12Trough['temperature-125cm'],'k')
put_axis('125cm', 'Trough')

plt.subplot(339)
plt.plot(Time[start_time:], DataObservedTrough['temperature-150cm'][start_time:],'r',label='Observed (hourly)')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-150cm'],'g',label='Observed (daily)')
plt.plot(TimeSim11, DataSim11Trough['temperature-150cm'],'k',label='Simulated')
plt.plot(TimeSim12, DataSim12Trough['temperature-150cm'],'k')
put_axis('150cm', 'Trough')

plt.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))

plt.tight_layout()
if save == True:
    plt.savefig(outfile+'temp-Trough%s-%s.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

In [24]:
# Polygon Rim
fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()

plt.subplot(331)
plt.plot(Time[start_time:], DataObservedRim['temperature-5cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-5cm'],'g-')
plt.plot(TimeSim11, DataSim11Rim['temperature-5cm'],'k')
plt.plot(TimeSim12, DataSim12Rim['temperature-5cm'],'k')
put_axis('5cm', 'Rim')

plt.subplot(332)
plt.plot(Time[start_time:], DataObservedRim['temperature-10cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-10cm'],'g-')
plt.plot(TimeSim11, DataSim11Rim['temperature-10cm'],'k')
plt.plot(TimeSim12, DataSim12Rim['temperature-10cm'],'k')
put_axis('10cm', 'Rim')

plt.subplot(333)
plt.plot(Time[start_time:], DataObservedRim['temperature-20cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-20cm'],'g-')
plt.plot(TimeSim11, DataSim11Rim['temperature-20cm'],'k')
plt.plot(TimeSim12, DataSim12Rim['temperature-20cm'],'k')
put_axis('20cm', 'Rim')

plt.subplot(334)
plt.plot(Time[start_time:], DataObservedRim['temperature-40cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-40cm'],'g-')
plt.plot(TimeSim11, DataSim11Rim['temperature-40cm'],'k')
plt.plot(TimeSim12, DataSim12Rim['temperature-40cm'],'k')
put_axis('40cm', 'Rim')

plt.subplot(335)
plt.plot(Time[start_time:], DataObservedRim['temperature-60cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-60cm'],'g-')
plt.plot(TimeSim11, DataSim11Rim['temperature-60cm'],'k')
plt.plot(TimeSim12, DataSim12Rim['temperature-60cm'],'k')
put_axis('60cm', 'Rim')

plt.subplot(336)
plt.plot(Time[start_time:], DataObservedRim['temperature-80cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-80cm'],'g-')
plt.plot(TimeSim11, DataSim11Rim['temperature-80cm'],'k')
plt.plot(TimeSim12, DataSim12Rim['temperature-80cm'],'k')
put_axis('80cm', 'Rim')

plt.subplot(337)
plt.plot(Time[start_time:], DataObservedRim['temperature-100cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-100cm'],'g-')
plt.plot(TimeSim11, DataSim11Rim['temperature-100cm'],'k')
plt.plot(TimeSim12, DataSim12Rim['temperature-100cm'],'k')
put_axis('100cm', 'Rim')

plt.subplot(338)
plt.plot(Time[start_time:], DataObservedRim['temperature-125cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-125cm'],'g-')
plt.plot(TimeSim11, DataSim11Rim['temperature-125cm'],'k')
plt.plot(TimeSim12, DataSim12Rim['temperature-125cm'],'k')
put_axis('125cm', 'Rim')

plt.subplot(339)
plt.plot(Time[start_time:], DataObservedRim['temperature-150cm'][start_time:],'r',label='Observed (hourly)')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-150cm'],'g', label='Observed (daily)')
plt.plot(TimeSim11, DataSim11Rim['temperature-150cm'],'k', label='Simulated')
plt.plot(TimeSim12, DataSim12Rim['temperature-150cm'],'k')
put_axis('150cm', 'Rim')

plt.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))

plt.tight_layout()
if save == True:
    plt.savefig(outfile+'temp-Rim%s-%s.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

In [ ]:
#simulation = 'CrunB12R'
simulation = 'CrunB42R12'
infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/surface-water_level-C37.dat"%simulation
Data37 = rd.ReadSingleFile(infile)
Data37['time'] = [d/86400. for d in Data37['time']] 
Data37['data'] = [d - 4.83541*0 for d in Data37['data']]
print (infile)
#infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/CrunB3R/surface-water_level-C37-ngbr.dat"
#Data37N = rd.ReadSingleFile(infile)
#Data37N['time'] = [d/86400. for d in Data37N['time']] 
#Data37N['data'] = [d - 4.83541 for d in Data37N['data']]

infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/surface-water_level-C40.dat"%simulation
Data40 = rd.ReadSingleFile(infile)
Data40['time'] = [d/86400. for d in Data40['time']] 
Data40['data'] = [d - 4.87831*0 for d in Data40['data']]

In [ ]:
import plot_water_table2D
times, wt, wt_ss, wt_bottom,thaw_depth = plot_water_table2D.water_table('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/'%simulation)

In [ ]:
lower_point = 4.8284937 
lower_point=  4.80342948 # new mesh
lower_point = 4.81031227 # new smooth mesh

In [ ]:
cellid = 36
cellid = 44 # new mesh
metdata = rd.GetSurfVarFromVis('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/visdump_surface_data.h5'%simulation, 'surface-precipitation_rain', cellid=cellid)
surftemp = rd.GetSurfVarFromVis('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/visdump_surface_data.h5'%simulation, 'surface-temperature', cellid=cellid)
#plt.plot(metdata['surface-precipitation_rain'])
#plt.show()

In [ ]:
rain = [d *100*900./0.02500 for d in metdata['surface-precipitation_rain']]
E1 = 4.72  + 0.075 * np.exp(rain)
#plt.plot(Data37['time'], Data37['data'], 'k', label='Simulated')
#plt.plot(DataC37['time'], DataC37['WL'],'r', label='Observed (C37)')

#plt.plot(Data37['time'],E1,'*')
#plt.show()
#simulation1 = 'CrunB30R79'
#times1, wt1, wt_ss1, wt_bottom1,thaw_depth1 = plot_water_table2D.water_table('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/'%simulation1)

In [ ]:
Data40 =dict()
time = [t*365.25 for t in times]
#time = [t for t in times]
Data40['time'] = time
Data40['wt'] = wt
Data40['wt_ss'] = wt_ss
Data40['wt_bottom'] = wt_bottom
print (times[0:3])
        
D1 = []
T1 = []
for i,d in enumerate(Data40['wt']):
    if d != 0:
        D1.append(d + lower_point)
        T1.append(Data40['time'][i])
        
D2 = []
T2 = []
for i,d in enumerate(Data40['wt']):
    D2.append(d + lower_point)
    T2.append(Data40['time'][i])
    
#water table bottom
D3 = []
T3 = []
for i,d in enumerate(Data40['wt_bottom']):
    D3.append(d + lower_point)
    T3.append(Data40['time'][i])
TD = []
for d in thaw_depth:
    TD.append(d + lower_point)   

"""
# Second simulation
Data401 =dict()
time1 = [t*365.25 for t in times]
Data401['time'] = time1
Data401['wt'] = wt1
Data401['wt_ss'] = wt_ss1
Data401['wt_bottom'] = wt_bottom1
        
D11 = []
T11 = []
for i,d in enumerate(Data401['wt']):
    if d != 0:
        D11.append(d + lower_point)
        T11.append(Data401['time'][i])
        
    
#water table bottom
D31 = []
T31 = []
for i,d in enumerate(Data401['wt_bottom']):
    D31.append(d + lower_point)
    T31.append(Data401['time'][i])
TD1 = []
for d in thaw_depth1:
    TD1.append(d + lower_point) 
"""

In [ ]:
outpath = '/Users/ajc/AllPostProcessData/BarrowTransect/Aug13-NewTC/'
fig, axs = plt.subplots(1,2,figsize=(10,4))
plt.subplot(121)
plt.plot(Data37['time'], Data37['data'], 'k', label='Simulated')
plt.plot(DataC37['time'], DataC37['WL'],'r', label='Observed (C37)')
plt.axhline(y=4.72)
#plt.plot(Data37['time'], Data37N['data'], 'm')
plt.xlabel('Day')
plt.ylabel('Water level')
plt.ylim((4.4,5.0))
plt.legend()


print (len(DataC39['time']), len(DataC39['WL']))
ax1 = plt.subplot(122)

plt.plot(Data40['time'], TD,'b', label='Thaw depth')
#plt.plot(Data401['time'], TD1,'c', label='Thaw depth (uf TC)') # 2nd simulation
plt.plot(DataC40['time'], DataC40['WL'],'r', label='Observed (C40)')
plt.plot(DataC39['time'], DataC39['WL'],'m', label='Observed (C39)')
plt.axhline(y=lower_point)
plt.plot(T1, D1,'k.',markersize='2.5',label='Simulated')
#plt.plot(T11, D11,'c.',markersize='2.5',label='Simulated (uf TC)') #2nd simulation


#plt.plot(T3, D3,'c',markersize='2.5',label='WT bottom')

#plt.plot(DataC41['time'], DataC41['WL'],'g', label='Observed (C41)')
#plt.plot(DataC42['time'], DataC42['WL'],'c', label='Observed (C42)')
#plt.plot(DataC43['time'], DataC43['WL'],'k', label='Observed (C43)')



plt.xlabel('Day')
plt.ylabel('Water level')
plt.legend(loc='lower left',fontsize=6)
plt.ylim((4.4,5.0))

ax2 = ax1.twinx()
rain = [d*1.e7 for d in metdata['surface-precipitation_rain']]
#temp = [d*1.e0 for d in surftemp['surface-temperature']]
ax2.plot(Data40['time'],rain,'y')
ax2.set_ylabel('Precipitation Rain [m /s] x 1.0e-7',color='y')
ax2.tick_params('y', colors='y')
ax2.set_ylim([-10.0, 2])


plt.savefig(outpath+'%s-transectB-C40-C39.pdf'%simulation, dpi=200)
#plt.savefig(outpath+'%s-transectB-C40-C39-TC-comp.pdf'%simulation, dpi=200)
plt.show()



In [ ]:
# Area B
fig,axs = plt.subplots(1,2, figsize=(14,6))
#fig.subplots_adjust(hsapce=0.2,wspace=0.2)
plt.subplot(121)
infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaB/Brun2R/surface-water_level-B64.dat"
Data64 = rd.ReadSingleFile(infile)
z_sim = 5.13042
Data64['time'] = [d/86400. for d in Data64['time']]
Data64['data'] = [(d - z_sim)*1 for d in Data64['data']]

ex = pd.ExcelFile('/Users/ajc/Desktop/ATS/barrow-polygon-data/water_table/NGEE_wells/RevisedNEAR-FINALWaterLevels-NGEESiteB-AC 01_28_2015.xlsx')
#print (ex.sheet_names)
DataB64EX = ex.parse(sheet_name='B64', skiprows=4)
print (DataB64EX.keys())
#DataB64EX = np.array(DataB64EX['Water Level 2013'])
#print (DataB64)
DataB64 = dict()
r = len(DataB64EX['Water Level 2013'])
DataB64['water_level'] = [(d - 4.99)*1 for d in DataB64EX[u'Water Level 2013']]
DataB64['time'] = [(t-time_origin).total_seconds()/86400. for t in DataB64EX['Date2013']]

l= len(DataB64)
plt.plot(Data64['time'], Data64['data'], 'k', label='Simulated')
#print (len(DataB64), len(Data['data'][:l]))
plt.plot(DataB64['time'], DataB64['water_level'],'r', label='Observed B64')
plt.xlabel('Days')
plt.ylabel('Water Depth [m]')
plt.legend()

# B67

plt.subplot(122)
infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaB/Brun2R/surface-water_level-B67.dat"
Data67 = rd.ReadSingleFile(infile)
z_sim = 5.50947
Data67['time'] = [d/86400. for d in Data67['time']]
Data67['data'] = [(d - z_sim)*1 for d in Data67['data']]

ex = pd.ExcelFile('/Users/ajc/Desktop/ATS/barrow-polygon-data/water_table/NGEE_wells/RevisedNEAR-FINALWaterLevels-NGEESiteB-AC 01_28_2015.xlsx')
print (ex.sheet_names)
DataB67EX = ex.parse(sheet_name='B67', skiprows=4)

#DataB67['water_level'] = np.array(DataB67EX['Water Level 2013'])
DataB67= dict()
DataB67['water_level'] = [(d - 5.44)*1 for d in DataB67EX[u'Water Level 2013']]
DataB67['time'] = [(t - time_origin).total_seconds()/86400. for t in DataB67EX['Date2013']]
print (DataB67['water_level'][:5], DataB67['time'][:5])
plt.plot(Data67['time'], Data67['data'], 'k', label='Simulated')
plt.plot(DataB67['time'], DataB67['water_level'],'r', label='Observed B67')
plt.xlabel('Days')
plt.ylabel('Water Depth [m]')
#plt.legend()

#plt.savefig('polygonAreaB-B64.pdf', dpi=200)
plt.show()
"""
for i in range(len(DataB67['water_level'])):
    if DataB67['water_level'][i] is not 'nan':
        print (i, DataB67['time'][i], DataB67['water_level'][i])
"""